## A <-> B reaction, with 1st-order kinetics in both directions,
### taken to equilibrium

Diffusion not done

LAST REVISED: Sep. 21, 2022

In [1]:
# Extend the sys.path variable, to contain the project's root directory
import set_path
set_path.add_ancestor_dir_to_syspath(3)  # The number of levels to go up 
                                         # to reach the project's home, from the folder containing this notebook

Added 'D:\Docs\- MY CODE\BioSimulations\life123-Win7' to sys.path


In [5]:
from experiments.get_notebook_info import get_notebook_basename

from modules.chemicals.chemicals import Chemicals as chem
from modules.reactions.reactions import Reactions
from life_2D.bio_sim_2d import BioSim2D

import plotly.express as px
from modules.html_log.html_log import HtmlLog as log
from modules.visualization.graphic_log import GraphicLog

In [3]:
# Initialize the HTML logging
log_file = get_notebook_basename() + ".log.htm"    # Use the notebook base filename for the log file
# Set up the use of some specified graphic (Vue) components
GraphicLog.config(filename=log_file,
                  components=["vue_cytoscape_1"],
                  extra_js="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.21.2/cytoscape.umd.js")

-> Output will be LOGGED into the file 'reaction_1.log.htm'


In [6]:
# Initialize the system
chem_data = chem(names=["A", "B"])     # NOTE: Diffusion not done

rxn = Reactions(chem_data)

# Reaction A <-> B , with 1st-order kinetics in both directions
rxn.add_reaction(reactants=["A"], products=["B"], forward_rate=3., reverse_rate=2.)

bio = BioSim2D(n_bins=(3,4), chem_data=chem_data, reactions=rxn)

bio.set_bin_conc_all_species(bin_x=0, bin_y=0, conc_list=[10.,50.])
bio.set_bin_conc_all_species(bin_x=0, bin_y=1, conc_list=[20.,35.])
bio.set_bin_conc_all_species(bin_x=2, bin_y=3, conc_list=[5.,100.])

bio.describe_state()

SYSTEM STATE at Time t = 0:
Species `A`:
[[10. 20.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  5.]]
Species `B`:
[[ 50.  35.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0. 100.]]


In [7]:
rxn.describe_reactions()

Number of reactions:  1
0: A <-> B  (Rf = 3.0 / Rb = 2.0)


In [8]:
# Send the plot to the HTML log file
graph_data = rxn.prepare_graph_network()
GraphicLog.export_plot(graph_data, "vue_cytoscape_1")


 {'reactants': [(1, 0, 1)], 'products': [(1, 1, 1)], 'Rf': 3.0, 'Rb': 2.0} 

[GRAPHIC ELEMENT SENT TO LOG FILE `reaction_1.log.htm`]


## First step

In [9]:
# First step (NOTE: here we're using a lower-level function that doesn't update the system state;
#                   it only computes the delta_reactions array)
bio.reaction_step(delta_time=0.1)
print("bio.delta_reactions:\n", bio.delta_reactions)

bio.delta_reactions:
 [[[  7.    1.    0.    0. ]
  [  0.    0.    0.    0. ]
  [  0.    0.    0.   18.5]]

 [[ -7.   -1.    0.    0. ]
  [  0.    0.    0.    0. ]
  [  0.    0.    0.  -18.5]]]


In [10]:
bio.system += bio.delta_reactions       # Matrix operation to update all the concentrations
bio.system_time += 0.1

bio.describe_state()

SYSTEM STATE at Time t = 0.1:
Species `A`:
[[17.  21.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.  23.5]]
Species `B`:
[[43.  34.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.  81.5]]


## Second step

In [11]:
# NOTE: now, we're using a highel-level function that also updates the system state
bio.react(time_step=0.1, n_steps=1)
bio.describe_state()

SYSTEM STATE at Time t = 0.2:
Species `A`:
[[20.5  21.5   0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.   32.75]]
Species `B`:
[[39.5  33.5   0.    0.  ]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.   72.25]]


## Many more steps, to equilibrium

In [12]:
bio.react(time_step=0.1, n_steps=8)
bio.describe_state()

SYSTEM STATE at Time t = 0.9999999999999999:
Species `A`:
[[23.98632812 21.99804688  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         41.96386719]]
Species `B`:
[[36.01367188 33.00195312  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         63.03613281]]


In [13]:
bio.react(time_step=0.1, n_steps=10)
bio.describe_state()

SYSTEM STATE at Time t = 2.0000000000000004:
Species `A`:
[[23.99998665 21.99999809  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         41.99996471]]
Species `B`:
[[36.00001335 33.00000191  0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.         63.00003529]]


### The system has now reached equilibrium
### in individual bins, which remain separate because we're NOT doing diffusion in this experiment

Verify the equilibrium in each of the active bins

In [14]:
bio.all_reactions.is_in_equilibrium(rxn_index=0, conc={"A": 23.99998665, "B": 36.00001335})

Ratio of forward/reverse reaction rates: 1.5
Ratio of reactant/product concentrations, adjusted for reaction orders: 1.5000013906257736
    [B] / [A]


True

In [15]:
bio.all_reactions.is_in_equilibrium(rxn_index=0, conc={"A": 21.99999809, "B": 33.00000191})

Ratio of forward/reverse reaction rates: 1.5
Ratio of reactant/product concentrations, adjusted for reaction orders: 1.5000002170454736
    [B] / [A]


True

In [16]:
bio.all_reactions.is_in_equilibrium(rxn_index=0, conc={"A": 41.99996471, "B": 63.00003529}, explain=False)

True